In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
classes = ["Balcony", "Bar", "Bathroom", "Bedroom", "Business Centre", "Dining room", "Exterior",
           "Gym", "Living room", "Lobby", "Patio", "Pool", "Restaurant", "Sauna", "Spa"]


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
num_epochs = 20
batch_size = 32
learning_rate = 0.001


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

In [8]:
dataset = ImageFolder("Dataset/", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Train Size : " + str(train_size));
print("Test Size : " + str(test_size))

Train Size : 59196
Test Size : 14799


In [9]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [13]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = torchvision.models.mobilenet_v2(pretrained=True)
        num_features = self.features.classifier[1].in_features
        self.features.classifier[1] = nn.Linear(num_features, len(classes))
    
    def forward(self, x):
        x = self.features(x)
        return x

model = CNNModel().to(device)

S:\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Shranay/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [15]:
total_step = len(train_loader)
print("Total Steps : " + str(total_step))

Total Steps : 1850


In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_correct = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Print training progress
        if (i + 1) == total_step:
            print(f"Epoch [{epoch + 1}/{num_epochs}]"
                  f"Train Loss: {loss.item():.4f}, Train Accuracy: {(total_correct / total_samples) * 100:.2f}%")
            
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
        
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
        
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
    
                print(f"Test Accuracy: {(100 * correct / total):.2f}%")
                print("")
        
        if epoch == num_epochs - 1:
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
    
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    # Collect predicted and true labels for classification report
                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            avg_precision = precision_score(all_labels, all_predicted, average='macro')
            avg_recall = recall_score(all_labels, all_predicted, average='macro')
            avg_f1_score = f1_score(all_labels, all_predicted, average='macro')

            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall: {avg_recall:.4f}")
            print(f"Average F1-Score: {avg_f1_score:.4f}")
            print("")

Epoch [1/20] Train Loss: 0.6563, Train Accuracy: 60.12%
Test Accuracy: 54.12%

Epoch [2/20] Train Loss: 0.4227, Train Accuracy: 67.85%
Test Accuracy: 60.76%

Epoch [3/20] Train Loss: 0.3491, Train Accuracy: 73.23%
Test Accuracy: 64.89%

Epoch [4/20] Train Loss: 0.2376, Train Accuracy: 77.94%
Test Accuracy: 71.32%

Epoch [5/20] Train Loss: 0.2148, Train Accuracy: 82.78%
Test Accuracy: 74.55%

Epoch [6/20] Train Loss: 0.1763, Train Accuracy: 86.83%
Test Accuracy: 77.86%

Epoch [7/20] Train Loss: 0.1345, Train Accuracy: 89.27%
Test Accuracy: 82.34%

Epoch [8/20] Train Loss: 0.1063, Train Accuracy: 91.34%
Test Accuracy: 86.21%

Epoch [9/20] Train Loss: 0.0911, Train Accuracy: 92.12%
Test Accuracy: 90.01%

Epoch [10/20] Train Loss: 0.0935, Train Accuracy: 90.78%
Test Accuracy: 89.62%

Epoch [11/20] Train Loss: 0.0972, Train Accuracy: 92.65%
Test Accuracy: 91.56%

Epoch [12/20] Train Loss: 0.0906, Train Accuracy: 91.18%
Test Accuracy: 90.81%

Epoch [13/20] Train Loss: 0.0952, Train Accuracy:

In [21]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {(100 * correct / total):.2f}%")

Validation Accuracy: 92.68%


In [20]:
model_path = "mobilenet.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved at: {model_path}")

Model saved at: mobilenet.pth
